This is the test code for checking the speed of neighbor_list ase vs matscipy vs torch_geometric vs pymatgen

In [32]:
from ase.io import read
from ase.neighborlist import primitive_neighbor_list # ase
from matscipy.neighbours import neighbour_list # matscipy
# torch_geometric
import torch
from torch_geometric.data import Data
from torch_geometric.transforms import RadiusGraph 

# pymatgen
from pymatgen.core import Structure
from pymatgen.analysis.local_env import CutOffDictNN
from pymatgen.analysis.graphs import StructureGraph

import sys
import numpy as np
# from mace.data.neighborhood import get_neighborhood
from sevenn.train.dataload import unlabeled_atoms_to_graph
import matplotlib.pyplot as plt

In [113]:
atoms = read('POSCAR_Li28La12Zr8O48')
atoms = read('Li20Ge2P4S24.cif') # 1714
atoms = read('Li20Ge2P4S24_112.cif') # 3428
# atoms = read('Li20Ge2P4S24_122.cif') # 6856
# atoms = read('Li20Ge2P4S24_222.cif') # 13712
pos = atoms.get_positions()
cell = np.array(atoms.get_cell())
cutoff = 6.0
pbc = atoms.get_pbc()

## Time check
---
### ASE

In [84]:
%%timeit -n 30
# ase.neighborlist
edge_src, edge_dst, edge_vec, shifts = primitive_neighbor_list(
        'ijDS', pbc, cell, pos, cutoff, self_interaction=False
    )

29.6 ms ± 832 µs per loop (mean ± std. dev. of 7 runs, 30 loops each)


### Matscipy

In [31]:
%%timeit -n 50
# matscipy.neighbours
edge_src, edge_dst, edge_vec, shifts = neighbour_list(
        quantities="ijDS",
        pbc=pbc,
        cell=cell,
        positions=pos,
        cutoff=5.0,
        # self_interaction=True,  # we want edges from atom to itself in different periodic images
        # use_scaled_positions=False,  # positions are not scaled positions
    )

672 µs ± 96.6 µs per loop (mean ± std. dev. of 7 runs, 50 loops each)


## Identical Graph?
---

### ASE

In [114]:
edge_src_ase, edge_dst_ase, edge_vec_ase, shifts_ase = primitive_neighbor_list(
        'ijDS', pbc, cell, pos, cutoff, self_interaction=False
    )

In [115]:
print(edge_src_ase, len(edge_src_ase))
print(edge_dst_ase, len(edge_dst_ase))
print(edge_vec_ase, len(edge_vec_ase))
print(shifts_ase, len(shifts_ase))

[ 0  0  0 ... 99 99 99] 6204
[46 32 27 ... 69 73 74] 6204
[[ 4.07496196  3.37574619  1.38541891]
 [ 0.95854456  3.64671974  3.93133256]
 [ 2.47725817  5.05887571 -0.68376052]
 ...
 [ 3.12411815  1.05943926  3.40697001]
 [ 3.99234212  1.76833204 -1.11721213]
 [ 2.15969948  0.16397909 -0.91314667]] 6204
[[0 1 0]
 [0 1 0]
 [0 1 0]
 ...
 [0 0 0]
 [0 0 0]
 [0 0 0]] 6204


### Matscipy

In [116]:
edge_src_mat, edge_dst_mat, edge_vec_mat, shifts_mat = neighbour_list(
        quantities="ijDS",
        pbc=pbc,
        cell=cell,
        positions=pos,
        cutoff=cutoff
        # self_interaction=True,  # we want edges from atom to itself in different periodic images
        # use_scaled_positions=False,  # positions are not scaled positions
    )

In [117]:
print(edge_src_mat, len(edge_src_mat))
print(edge_dst_mat, len(edge_dst_mat))
print(edge_vec_mat, len(edge_vec_mat))
print(shifts_mat, len(shifts_mat))

[ 0  0  0 ... 99 99 99] 6204
[58 98 57 ...  9 22 27] 6204
[[-4.61062832  0.10045171 -2.27079464]
 [-5.096045   -1.99453914 -2.29656295]
 [-1.30016726 -4.45452978 -1.55589879]
 ...
 [-0.00974946  0.61694636  5.46864382]
 [-2.90327695 -1.06370515  5.13515784]
 [ 2.8029091  -0.44433024  5.03769901]] 6204
[[-1  0 -1]
 [-1  0 -1]
 [ 0  0 -1]
 ...
 [-1  0  1]
 [-1  0  1]
 [ 0  0  1]] 6204


### Torch_geometric

In [51]:
pos_tensor = torch.tensor(pos, dtype=torch.float)
data = Data(pos=pos_tensor)
data = RadiusGraph(5.0)(data)

In [52]:
structure = Structure.from_file("POSCAR_Li28La12Zr8O48")
cutoff = 5.0

# Use a cutoff-based nearest neighbors strategy
nn_strategy = CutOffDictNN(cut_off_dict={"Li": cutoff, "La": cutoff, "Zr": cutoff, "O": cutoff})
# structure_graph = StructureGraph.with_local_env_strategy(structure, nn_strategy)

# Extract edges (source, destination) and edge vectors
# edges = structure_graph.graph.edges(data=True)
# edge_src, edge_dst, edge_vec = [], [], []

# for u, v, d in edges:
#     edge_src.append(u)
#     edge_dst.append(v)
#     edge_vec.append(d["to_jimage"])  # Fractional lattice vector

# print("Pymatgen edges:", len(edge_src))

ValueError: not enough values to unpack (expected 2, got 1)

## Compare graphs
---

In [118]:
def compare_graphs(edge_vec_1, edge_src_1, edge_dst_1, 
                   edge_vec_2, edge_src_2, edge_dst_2):
    """
    Compare two graphs (ASE and matscipy) based on edge vectors, sources, and destinations.

    Parameters:
        edge_vec_1: np.ndarray
            Edge vectors from ASE.
        edge_src_1: np.ndarray
            Source nodes from ASE.
        edge_dst_1: np.ndarray
            Destination nodes from ASE.
        edge_vec_2: np.ndarray
            Edge vectors from matscipy.
        edge_src_2: np.ndarray
            Source nodes from matscipy.
        edge_dst_2: np.ndarray
            Destination nodes from matscipy.
    """
    # Sort edge_vec and get sorting indices
    # sorted_indices_ase = np.lexsort(edge_vec_1.T)
    # sorted_indices_mat = np.lexsort(edge_vec_2.T)
    sorted_indices_ase = np.lexsort((edge_vec_1[:, 2], edge_vec_1[:, 1], edge_vec_1[:, 0]))
    sorted_indices_mat = np.lexsort((edge_vec_2[:, 2], edge_vec_2[:, 1], edge_vec_2[:, 0]))

    # Sorted edge_vec
    sorted_vec_ase = edge_vec_1[sorted_indices_ase]
    sorted_vec_mat = edge_vec_2[sorted_indices_mat]

    # Compare sorted edge_vec
    are_equivalent = np.allclose(sorted_vec_ase, sorted_vec_mat, atol=1e-6)
    print("Are the edge_vec matrices row-equivalent?", are_equivalent)

    if not are_equivalent:
        # Find rows that differ
        differences = np.abs(sorted_vec_ase - sorted_vec_mat)
        differing_rows = np.where(np.max(differences, axis=1) > 1e-6)[0]
        same_rows = np.where(np.max(differences, axis=1) <= 1e-6)[0]

        print("\nRows that differ (exceeding atol):")
        for row in differing_rows:
            print(f"ASE: {sorted_vec_ase[row]}, matscipy: {sorted_vec_mat[row]}, difference: {differences[row]}, nodes1: {edge_src_1[sorted_indices_ase[row]], edge_dst_1[sorted_indices_ase[row]]}, nodes2: {edge_src_2[sorted_indices_mat[row]], edge_dst_2[sorted_indices_mat[row]]}")

        for row in same_rows:
            print(f"ASE: {sorted_vec_ase[row]}, matscipy: {sorted_vec_mat[row]}, difference: {differences[row]}, nodes1: {edge_src_1[sorted_indices_ase[row]], edge_dst_1[sorted_indices_ase[row]]}, nodes2: {edge_src_2[sorted_indices_mat[row]], edge_dst_2[sorted_indices_mat[row]]}")


    if are_equivalent:
        # Match edge_src and edge_dst for ASE
        sorted_src_1 = edge_src_1[sorted_indices_ase]
        sorted_dst_1 = edge_dst_1[sorted_indices_ase]

        # Match edge_src and edge_dst for matscipy
        sorted_src_2 = edge_src_2[sorted_indices_mat]
        sorted_dst_2 = edge_dst_2[sorted_indices_mat]

        # Check if the sources and destinations align
        sources_match = np.array_equal(sorted_src_1, sorted_src_2)
        destinations_match = np.array_equal(sorted_dst_1, sorted_dst_2)

        # Check if the sources and destinations align
        sources_match = np.array_equal(sorted_src_1, sorted_src_2)
        destinations_match = np.array_equal(sorted_dst_1, sorted_dst_2)

        if sources_match and destinations_match:
            print("Two graphs have the same node indices.")
        else:
            print("The graphs have different node indices.")

        # Print edges side by side for comparison
        print("ASE (src, dst) | matscipy (src, dst)")
        print("-------------------------------------")
        for ase_edge, mat_edge in zip(zip(sorted_src_1, sorted_dst_1), zip(sorted_src_2, sorted_dst_2)):
            print(f"{ase_edge} | {mat_edge}")
    else:
        print("The graphs are not equivalent.")

def compare_graphs_ver2(edge_vec_1, edge_src_1, edge_dst_1, shift_1,
                   edge_vec_2, edge_src_2, edge_dst_2, shift_2):
    """
    Compare two graphs (ASE and matscipy) based on edge vectors, sources, and destinations.

    Parameters:
        edge_vec_1: np.ndarray
            Edge vectors from ASE.
        edge_src_1: np.ndarray
            Source nodes from ASE.
        edge_dst_1: np.ndarray
            Destination nodes from ASE.
        edge_vec_2: np.ndarray
            Edge vectors from matscipy.
        edge_src_2: np.ndarray
            Source nodes from matscipy.
        edge_dst_2: np.ndarray
            Destination nodes from matscipy.
    """
    # Sort edge_vec and get sorting indices
    # sorted_indices_ase = np.lexsort(edge_vec_1.T)
    # sorted_indices_mat = np.lexsort(edge_vec_2.T)
    sorted_indices_ase = np.lexsort((edge_vec_1[:, 2], edge_vec_1[:, 1], edge_vec_1[:, 0]))
    sorted_indices_mat = np.lexsort((edge_vec_2[:, 2], edge_vec_2[:, 1], edge_vec_2[:, 0]))

    # Sorted edge_vec
    sorted_vec_ase = edge_vec_1[sorted_indices_ase]
    sorted_vec_mat = edge_vec_2[sorted_indices_mat]

    # Sorted shifts
    sorted_shifts_ase = shift_1[sorted_indices_ase]
    sorted_shifts_mat = shift_2[sorted_indices_mat]

    for i in range(len(sorted_vec_ase)):
        if np.allclose(sorted_vec_ase[i], sorted_vec_mat[i], atol=1e-6):
            # print(f"ASE: {sorted_vec_ase[i]}, matscipy: {sorted_vec_mat[i]}")
            # print(f"ASE shift: {sorted_shifts_ase[i]}, matscipy shift: {sorted_shifts_mat[i]}")
            # print()
            if not np.array_equal(sorted_shifts_ase[i], sorted_shifts_mat[i]):
                print(f"ASE: {sorted_vec_ase[i]}, matscipy: {sorted_vec_mat[i]}")
                print(f"ASE shift: {sorted_shifts_ase[i]}, matscipy shift: {sorted_shifts_mat[i]}")
                print(f'ASE node: {edge_src_1[sorted_indices_ase[i]], edge_dst_1[sorted_indices_ase[i]]}, matscipy node: {edge_src_2[sorted_indices_mat[i]], edge_dst_2[sorted_indices_mat[i]]}')
                print()

    

In [119]:
compare_graphs(edge_vec_ase, edge_src_ase, edge_dst_ase, edge_vec_mat, edge_src_mat, edge_dst_mat)

Are the edge_vec matrices row-equivalent? True
The graphs have different node indices.
ASE (src, dst) | matscipy (src, dst)
-------------------------------------
(24, 33) | (24, 33)
(74, 83) | (74, 83)
(78, 62) | (78, 62)
(28, 12) | (28, 12)
(65, 87) | (65, 87)
(15, 37) | (15, 37)
(41, 17) | (41, 17)
(91, 67) | (91, 67)
(46, 1) | (46, 1)
(96, 51) | (96, 51)
(38, 11) | (88, 61)
(88, 61) | (38, 11)
(99, 74) | (99, 74)
(49, 24) | (49, 24)
(72, 30) | (72, 30)
(22, 80) | (22, 80)
(36, 9) | (36, 9)
(86, 59) | (86, 59)
(27, 22) | (27, 22)
(77, 72) | (77, 72)
(39, 2) | (39, 2)
(89, 52) | (89, 52)
(81, 60) | (31, 10)
(31, 10) | (81, 60)
(66, 78) | (66, 78)
(16, 28) | (16, 28)
(80, 57) | (80, 57)
(30, 7) | (30, 7)
(26, 4) | (76, 54)
(76, 54) | (26, 4)
(57, 68) | (57, 68)
(7, 18) | (7, 18)
(85, 58) | (85, 58)
(35, 8) | (35, 8)
(71, 95) | (71, 95)
(21, 45) | (21, 45)
(83, 53) | (83, 53)
(33, 3) | (33, 3)
(62, 65) | (62, 65)
(12, 15) | (12, 15)
(64, 53) | (64, 53)
(14, 3) | (14, 3)
(12, 31) | (12, 

In [120]:
compare_graphs_ver2(edge_vec_ase, edge_src_ase, edge_dst_ase, shifts_ase, edge_vec_mat, edge_src_mat, edge_dst_mat, shifts_mat)

ASE: [-4.56488224  2.66888799 -1.58703412], matscipy: [-4.56488224  2.66888799 -1.58703412]
ASE shift: [-1  0 -1], matscipy shift: [-1  0  0]
ASE node: (27, 58), matscipy node: (77, 8)

ASE: [-4.56488224  2.66888799 -1.58703412], matscipy: [-4.56488224  2.66888799 -1.58703412]
ASE shift: [-1  0  0], matscipy shift: [-1  0 -1]
ASE node: (77, 8), matscipy node: (27, 58)

ASE: [-4.03703454  2.35133979 -1.5355026 ], matscipy: [-4.03703454  2.35133979 -1.5355026 ]
ASE shift: [-1  1 -1], matscipy shift: [-1  1  0]
ASE node: (0, 91), matscipy node: (50, 41)

ASE: [-4.03703454  2.35133979 -1.5355026 ], matscipy: [-4.03703454  2.35133979 -1.5355026 ]
ASE shift: [-1  1  0], matscipy shift: [-1  1 -1]
ASE node: (50, 41), matscipy node: (0, 91)

ASE: [-3.99153428 -1.91798129  1.95979466], matscipy: [-3.99153428 -1.91798129  1.95979466]
ASE shift: [-1  0  0], matscipy shift: [-1  0  1]
ASE node: (30, 70), matscipy node: (80, 20)

ASE: [-3.99153428 -1.91798129  1.95979466], matscipy: [-3.99153428 -1